##Install requirements packages

In [1]:
!pip install transformers
!pip install fastBPE
!pip install fairseq
# Install the vncorenlp python wrapper
!pip install vncorenlp
# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp310-cp310-linux_x86_64.whl size=806315 sha256=d9277b24f5245fd823b0f0231b03f478bce712d18b01eaeaed9668d0e0dbc4bb
  Stored in directory: /root/.cache/pip/wheels/13/5d/b9/4b8897941ebc9e8c6cc3f3ffd3ea5115731754269205098754
Successfully built fastBPE
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 76.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done

**Install VnCoreNLP**

In [2]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

**Install pre-trained PhoBERT**

In [3]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz

--2023-10-18 07:56:49--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.192.150.106, 54.192.150.98, 54.192.150.17, ...
Connecting to public.vinai.io (public.vinai.io)|54.192.150.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: ‘PhoBERT_base_transformers.tar.gz’

PhoBERT_base_transf 100%[===================>] 307.47M  67.9MB/s    in 4.6s    

2023-10-18 07:56:54 (66.4 MB/s) - ‘PhoBERT_base_transformers.tar.gz’ saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [4]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz

--2023-10-18 07:57:03--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.192.150.106, 54.192.150.98, 54.192.150.17, ...
Connecting to public.vinai.io (public.vinai.io)|54.192.150.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz’

PhoBERT_base_fairse 100%[===================>]   1.16G  58.9MB/s    in 19s     

2023-10-18 07:57:21 (63.3 MB/s) - ‘PhoBERT_base_fairseq.tar.gz’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


**Load Phobert**

In [5]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes',
    default="/content/PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("/content/PhoBERT_base_transformers/dict.txt")
#load mô hình

# Data preparing

In [7]:
import pandas as pd

label1 = pd.read_csv("labeled_sentences_1.csv", encoding='utf8', engine='python').dropna()
label2 = pd.read_csv("labeled_sentences_2.csv", encoding='utf8', engine='python').dropna()
label3 = pd.read_csv("labeled_sentences_3.csv", encoding='utf8', engine='python').dropna()
label4 = pd.read_csv("data_long_nguyen_et_al.csv", encoding='utf8', engine='python', names=["Sentences", "Sentiment"]).dropna()
label4['Sentiment'] = label4['Sentiment'] - 2
merge_data = pd.concat([label1, label2, label3, label4]).reset_index(drop=True)
merge_data['Sentiment'] = merge_data['Sentiment'] + 1

In [8]:
import re


train_text, train_labels = [], []


for i in merge_data.index:
    label = int(merge_data['Sentiment'][i])
    text = rdrsegmenter.tokenize(merge_data['Sentences'][i])
    text = ' '.join([' '.join(x) for x in text])

    train_text.append(text)
    train_labels.append(label)
    # thực hiện việc trích xuất và xử lý dữ liệu văn bản và nhãn từ một dataframe và lưu chúng vào danh sách để chuẩn bị cho quá trình huấn luyện mô hình

**Train test split**

In [9]:
from sklearn.model_selection import train_test_split

train_sents, val_sents, train_labels, val_labels = train_test_split(train_text, train_labels, test_size=0.1)

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 195

train_ids = []
for sent in train_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

val_ids = []
for sent in val_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_sent)

train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
#Mã hóa văn bản thành các chuỗi số sử dụng một từ điển vocab và mã hóa các subwords trong mỗi văn bản
#kq là hai danh sách train_ids và val_ids chứa các chuỗi số đã được chuẩn bị để đưa vào mô hình

In [11]:
train_masks = []
for sent in train_ids:
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

val_masks = []
for sent in val_ids:
    mask = [int(token_id > 0) for token_id in sent]

    val_masks.append(mask)

**DataLoader**

In [12]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)
#  chuẩn bị dữ liệu huấn luyện và kiểm tra thành các batch để sử dụng trong quá trình huấn luyện

# Train

In [13]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained("/content/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 3, output_hidden_states=False,)
BERT_SA = RobertaForSequenceClassification.from_pretrained( "/content/PhoBERT_base_transformers/model.bin", config=config)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/PhoBERT_base_transformers/model.bin and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
BERT_SA.cuda()
print('Done')

Done


In [15]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    F1_score = f1_score(pred_flat, labels_flat, average='macro')

    return accuracy_score(pred_flat, labels_flat), F1_score

In [ ]:
import random
import tqdm
device = 'cuda'
epochs = 20
max = 0

param_optimizer = list(BERT_SA.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=5e-7)


for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    BERT_SA.train()
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0

    for step, batch in tqdm.notebook.tqdm(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        BERT_SA.zero_grad()
        outputs = BERT_SA(b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask,
            labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()

        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
        train_accuracy += tmp_train_accuracy
        train_f1 += tmp_train_f1
        nb_train_steps += 1

        loss.backward()
        torch.nn.utils.clip_grad_norm_(BERT_SA.parameters(), 1.0)
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(" Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
    print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
    print(" Average training loss: {0:.4f}".format(avg_train_loss))
    BERT_SA.save_pretrained('/content/drive/MyDrive/Deep/modelbert/{}'.format(epoch_i+1))
    print("Running Validation...")
    BERT_SA.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0
    for batch in tqdm.notebook.tqdm(val_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = BERT_SA(b_input_ids,
            token_type_ids=None,
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)

            eval_accuracy += tmp_eval_accuracy
            eval_f1 += tmp_eval_f1
            nb_eval_steps += 1
    if max < eval_accuracy/nb_eval_steps:
      max = eval_accuracy/nb_eval_steps
      e = epoch_i
    print('logits:    {}'.format(np.argmax(logits, axis=1).flatten()))
    print('labels_id: {}'.format(label_ids))
    print(" Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))
print("Training complete!")
print('Best Valid acc : {}, epoch: {}'.format(max, e+1))
# huấn luyện mô hình BERT để thực hiện một tác vụ cụ thể và đánh giá hiệu suất của mô hình trên tập kiểm tra sau mỗi epoch

======== Epoch 1 / 20 ========
Training...


0it [00:00, ?it/s]

 Accuracy: 0.3627
 F1 score: 0.2888
 Average training loss: 1.0928
Running Validation...


  0%|          | 0/13 [00:00<?, ?it/s]

logits:    [2 2 1 1 1 2 2 2 2 2 2 1 2 2 2 2 1]
labels_id: [2 0 1 0 1 2 0 2 0 0 2 1 2 1 2 1 1]
 Accuracy: 0.3986
 F1 score: 0.3075
======== Epoch 2 / 20 ========
Training...


...

0it [00:00, ?it/s]

 Accuracy: 0.5252
 F1 score: 0.4596
 Average training loss: 1.0047
Running Validation...


  0%|          | 0/13 [00:00<?, ?it/s]

logits:    [2 2 0 1 0 2 2 1 2 1 2 0 2 1 2 1 2]
labels_id: [2 0 1 0 1 2 0 2 0 0 2 1 2 1 2 1 1]
 Accuracy: 0.5341
 F1 score: 0.4977
======== Epoch 13 / 20 ========
Training...


0it [00:00, ?it/s]

# Save model

In [ ]:
# Define the test input
test_input = "Đây là một câu văn để kiểm tra mô hình."

# Tokenize the test input
input_ids = tokenizer.encode(test_input, add_special_tokens=True)
input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

# Set the model in evaluation mode
BERT_SA.eval()

# Move the input tensor to the GPU if available
if torch.cuda.is_available():
    input_ids = input_ids.to('cuda')

# Perform inference on the test input
with torch.no_grad():
    outputs = BERT_SA(input_ids)

# Get the predicted class
predicted_class = torch.argmax(outputs.logits, dim=1).item()

# Print the results
print("Test Input: ", test_input)
print("Predicted Class (Label): ", predicted_class)

In [ ]:
!zip -r /content/model.zip/content/drive/MyDrive/Deep/modelbert/3

In [ ]:
# LSTM (Long Short-Term Memory) và GRU (Gated Recurrent Unit): Kiến trúc RNN (Recurrent Neural Network) như LSTM và GRU có thể được sử dụng để xử lý chuỗi dữ liệu văn bản. Chúng có khả năng hiểu ngữ cảnh và tạo ra biểu diễn chuỗi cho mỗi câu, sau đó sử dụng các lớp mạng nơ-ron bổ sung để phân loại tình cảm hoặc thái độ.

# CNN (Convolutional Neural Network): Mạng nơ-ron tích chập có thể được sử dụng để trích xuất đặc trưng từ văn bản. Điều này đặc biệt hữu ích khi bạn muốn xử lý cảm xúc trong các câu ngắn. CNN có thể tự động học các đặc trưng quan trọng trong văn bản.

# Mô hình Transformer cải tiến: Ngoài BERT, còn có các mô hình Transformer cải tiến khác như GPT (Generative Pre-trained Transformer) và RoBERTa. Các mô hình này đã được đào tạo trên nhiều tác vụ NLP và có khả năng hiểu cảm xúc trong văn bản.

# Mô hình FastText: Mô hình FastText dựa trên từ và n-gram có thể được sử dụng để biểu diễn văn bản và xử lý cảm xúc. Nó nhanh chóng và hiệu quả đối với các tác vụ nhỏ và ngắn.

# Mô hình BERT tiếng Việt khác: Ngoài PhoBERT, còn có các biến thể của BERT được đào tạo trên dữ liệu tiếng Việt khác như VinBERT. Chúng có thể được sử dụng cho việc nhận biết tình cảm và thái độ trong văn bản tiếng Việt.

# SVM (Support Vector Machine), Naive Bayes và các mô hình truyền thống khác: Không nhất thiết phải sử dụng mô hình mạng nơ-ron sâu. Các mô hình truyền thống như SVM và Naive Bayes có thể được sử dụng để phân loại tình cảm trong văn bản dựa trên các đặc trưng thủ công.

# Lựa chọn mô hình phụ thuộc vào tình huống cụ thể, kích thước dữ liệu, và yêu cầu hiệu suất. Đối với các tác vụ phức tạp và tập dữ liệu lớn, các mô hình như BERT thường mang lại hiệu suất tốt. Tuy nhiên, đối với tác vụ đơn giản và dữ liệu hạn chế, các mô hình truyền thống có thể hoạt động tốt.





